In [6]:
import os
from glob import glob

import pandas as pd
from dotenv import load_dotenv

from src.file_wrapper.meta_data import GeographieMeta, StationNameMeta
from src.file_wrapper.weather_data import Wind10MinutesData

load_dotenv()

META_DATA_FOLDER = os.path.join(os.getenv('PROJECT_DIR'), 'data','interim', 'station_meta' )
DATA_FOLDER = os.path.join(os.getenv('PROJECT_DIR'), 'data','interim', 'wind10_min' )
PARQUET_FOLDER = os.path.join(os.getenv('PROJECT_DIR'), 'data','interim', 'parquet' )

GET_WIND = True

In [7]:
%%time
geo_meta_data = []
name_meta_data = []
meta_data_folders = glob(os.path.join(META_DATA_FOLDER, '*'))

if not os.path.exists(PARQUET_FOLDER):
    os.makedirs(PARQUET_FOLDER)

for folder in meta_data_folders:
    geo_meta_data.append(GeographieMeta(folder))
    name_meta_data.append(StationNameMeta(folder))

geo_df = pd.concat([gm.data for gm in geo_meta_data])
name_df = pd.concat([nm.data for nm in name_meta_data])

geo_df.to_parquet(os.path.join(PARQUET_FOLDER, 'geo.parquet'))
name_df.to_parquet(os.path.join(PARQUET_FOLDER, 'name.parquet'))


CPU times: user 1.28 s, sys: 166 ms, total: 1.44 s
Wall time: 1.61 s


In [10]:
%%time
if GET_WIND:
    wind_data = []
    wind_data_folders = glob(os.path.join(DATA_FOLDER, '*'))
    
    print(f"Found {len(wind_data_folders)} folders")
    for i, folder in enumerate(wind_data_folders):
        wind_data.append(Wind10MinutesData(folder))
        
        
    wind_df = pd.concat([wd.data for wd in wind_data])
    wind_df = wind_df[~((wind_df['avg_direction'].isna()) & (wind_df['avg_speed'].isna()))]
    
    wind_df.to_parquet(os.path.join(PARQUET_FOLDER, 'wind.parquet'), engine='pyarrow')

Found 997 folders
CPU times: user 21min 49s, sys: 3min 17s, total: 25min 6s
Wall time: 33min 1s
